In [2]:
# ============================================================================
# Travelers UMC 2025 - Baseline Version 2
# Optimal 3-Model Ensemble for Subrogation Prediction
# ============================================================================
# Install missing packages if needed
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully!")

# Check and install required packages
packages_to_check = {
    'imbalanced-learn': 'imblearn',
    'optuna': 'optuna',
    'xgboost': 'xgboost'
}

for pip_name, import_name in packages_to_check.items():
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {pip_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name])
        print(f"{pip_name} installed successfully!")

# All Required Libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Data handling
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, classification_report

# Models
import xgboost as xgb
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV

# Imbalance handling
from imblearn.over_sampling import SMOTE

# Hyperparameter optimization
import optuna
from optuna.samplers import TPESampler

# Visualization (optional)
import matplotlib.pyplot as plt
import seaborn as sns

print("\n" + "="*80)
print("All libraries imported successfully!")
print(f"XGBoost version: {xgb.__version__}")
print("="*80)


Installing optuna...
optuna installed successfully!

All libraries imported successfully!
XGBoost version: 3.1.1


In [3]:
# ============================================================================
# STEP 1: Load Engineered Data
# ============================================================================
print("="*80)
print("LOADING ENGINEERED DATA")
print("="*80)

train_df = pd.read_csv('engineered_train.csv')
test_df = pd.read_csv('engineered_test.csv')
test_original = pd.read_csv('Data\\Testing_TriGuard.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# Drop non-feature columns
cols_to_drop = ['claim_date', 'year_of_born', 'vehicle_made_year', 
                'gender', 'living_status', 'claim_day_of_week', 'accident_site',
                'witness_present_ind', 'channel', 'vehicle_category', 'vehicle_color',
                'accident_type', 'in_network_bodyshop', 'zip_group']

train_df = train_df.drop(columns=[col for col in cols_to_drop if col in train_df.columns], errors='ignore')
test_df = test_df.drop(columns=[col for col in cols_to_drop if col in test_df.columns], errors='ignore')

# Separate features and target
X = train_df.drop(['subrogation'], axis=1, errors='ignore')
y = train_df['subrogation'].astype(int)
X_test = test_df.copy()

# Keep only numeric columns
X = X.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])

# Align columns between train and test
X_test = X_test.reindex(columns=X.columns, fill_value=0)

# Handle any remaining NaN/inf values
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

print(f"\nFinal feature count: {X.shape[1]}")
print(f"Train samples: {X.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"\nClass distribution:\n{y.value_counts(normalize=True)}")


LOADING ENGINEERED DATA
Train shape: (18000, 105)
Test shape: (12000, 104)

Final feature count: 90
Train samples: 18000
Test samples: 12000

Class distribution:
subrogation
0    0.771389
1    0.228611
Name: proportion, dtype: float64


In [4]:
# ============================================================================
# STEP 2: Handle Class Imbalance with SMOTE
# ============================================================================
print("\n" + "="*80)
print("APPLYING SMOTE FOR CLASS BALANCING")
print("="*80)

smote = SMOTE(sampling_strategy=0.5, random_state=42, k_neighbors=5)
X_resampled, y_resampled = smote.fit_resample(X, y)

print(f"Before SMOTE: {X.shape[0]} samples")
print(f"After SMOTE: {X_resampled.shape[0]} samples")
print(f"Class distribution after SMOTE:\n{pd.Series(y_resampled).value_counts(normalize=True)}")



APPLYING SMOTE FOR CLASS BALANCING
Before SMOTE: 18000 samples
After SMOTE: 20827 samples
Class distribution after SMOTE:
subrogation
0    0.666683
1    0.333317
Name: proportion, dtype: float64


In [5]:
# ============================================================================
# STEP 3: Feature Selection (Top Features)
# ============================================================================
print("\n" + "="*80)
print("FEATURE SELECTION")
print("="*80)

selector_model = xgb.XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    learning_rate=0.1,
    random_state=42, 
    eval_metric='auc',
    tree_method='hist'
)
selector_model.fit(X_resampled, y_resampled)

# Select top 150 features
selector = SelectFromModel(selector_model, max_features=150, threshold=-np.inf)
X_selected = selector.transform(X_resampled)
X_test_selected = selector.transform(X_test)

selected_features = X.columns[selector.get_support()].tolist()
print(f"Selected {len(selected_features)} features from {X.shape[1]} total features")
print(f"\nTop 20 features: {selected_features[:20]}")



FEATURE SELECTION
Selected 90 features from 90 total features

Top 20 features: ['email_or_tel_available', 'safety_rating', 'annual_income', 'high_education_ind', 'address_change_ind', 'zip_code', 'past_num_of_claims', 'liab_prct', 'policy_report_filed_ind', 'claim_est_payout', 'vehicle_price', 'vehicle_weight', 'age_of_DL', 'vehicle_mileage', 'claim_month', 'claim_year', 'driver_age', 'vehicle_age', 'vehicle_price_default', 'annual_income_log']


In [6]:
# ============================================================================
# STEP 4: Hyperparameter Optimization with Optuna
# ============================================================================
print("\n" + "="*80)
print("BAYESIAN HYPERPARAMETER OPTIMIZATION")
print("="*80)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Model 1: XGBoost Optimization
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 3),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 8),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 2),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 2),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 4),
        'random_state': 42,
        'eval_metric': 'auc',
        'tree_method': 'hist'
    }
    model = xgb.XGBClassifier(**params)
    scores = cross_val_score(model, X_selected, y_resampled, cv=cv, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

# Model 2: ExtraTrees Optimization
def objective_et(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 300, 800),
        'max_depth': trial.suggest_int('max_depth', 10, 25),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', 0.5, 0.7]),
        'bootstrap': True,
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced']),
        'random_state': 42,
        'n_jobs': -1
    }
    model = ExtraTreesClassifier(**params)
    scores = cross_val_score(model, X_selected, y_resampled, cv=cv, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

# Model 3: Logistic Regression Optimization
def objective_lr(trial):
    penalty = trial.suggest_categorical('penalty', ['l2', 'elasticnet'])
    C = trial.suggest_float('C', 0.01, 100, log=True)
    class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
    
    if penalty == 'elasticnet':
        l1_ratio = trial.suggest_float('l1_ratio', 0.1, 0.9)
        params = {
            'C': C,
            'penalty': penalty,
            'l1_ratio': l1_ratio,
            'solver': 'saga',
            'max_iter': 2000,
            'class_weight': class_weight,
            'random_state': 42
        }
    else:
        params = {
            'C': C,
            'penalty': penalty,
            'solver': 'lbfgs',
            'max_iter': 2000,
            'class_weight': class_weight,
            'random_state': 42
        }
    
    model = LogisticRegression(**params)
    scores = cross_val_score(model, X_selected, y_resampled, cv=cv, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

print("\nOptimizing XGBoost...")
study_xgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)
best_xgb_params = study_xgb.best_params
print(f"Best XGBoost AUC: {study_xgb.best_value:.5f}")

print("\nOptimizing ExtraTrees...")
study_et = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_et.optimize(objective_et, n_trials=50, show_progress_bar=True)
best_et_params = study_et.best_params
print(f"Best ExtraTrees AUC: {study_et.best_value:.5f}")

print("\nOptimizing Logistic Regression...")
study_lr = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_lr.optimize(objective_lr, n_trials=30, show_progress_bar=True)
best_lr_params = study_lr.best_params
print(f"Best Logistic Regression AUC: {study_lr.best_value:.5f}")

print("\n" + "="*80)
print("OPTIMIZATION COMPLETE")
print("="*80)


[I 2025-11-02 01:24:27,092] A new study created in memory with name: no-name-04a7fe92-1891-4892-af74-d9fae040b443



BAYESIAN HYPERPARAMETER OPTIMIZATION

Optimizing XGBoost...


Best trial: 0. Best value: 0.898858:   2%|▏         | 1/50 [00:07<05:44,  7.03s/it]

[I 2025-11-02 01:24:34,126] Trial 0 finished with value: 0.8988581176161583 and parameters: {'n_estimators': 874, 'max_depth': 10, 'learning_rate': 0.07259248719561363, 'subsample': 0.8795975452591109, 'colsample_bytree': 0.7468055921327309, 'gamma': 0.46798356100860794, 'min_child_weight': 1, 'reg_alpha': 1.7323522915498704, 'reg_lambda': 1.2022300234864176, 'scale_pos_weight': 3.1242177333881367}. Best is trial 0 with value: 0.8988581176161583.


Best trial: 0. Best value: 0.898858:   4%|▍         | 2/50 [00:11<04:33,  5.71s/it]

...

[I 2025-11-02 01:34:52,536] Trial 27 finished with value: 0.8008627985825015 and parameters: {'penalty': 'l2', 'C': 0.023190146464774648, 'class_weight': 'balanced'}. Best is trial 12 with value: 0.8053809218117539.


Best trial: 12. Best value: 0.805381:  97%|█████████▋| 29/30 [03:41<00:06,  6.33s/it]

[I 2025-11-02 01:34:56,403] Trial 28 finished with value: 0.7926758237739497 and parameters: {'penalty': 'l2', 'C': 0.33968762817977727, 'class_weight': 'balanced'}. Best is trial 12 with value: 0.8053809218117539.


Best trial: 12. Best value: 0.805381: 100%|██████████| 30/30 [04:02<00:00,  8.08s/it]

[I 2025-11-02 01:35:17,335] Trial 29 finished with value: 0.6239008510073673 and parameters: {'penalty': 'elasticnet', 'C': 10.092271935851178, 'class_weight': 'balanced', 'l1_ratio': 0.12177522111295758}. Best is trial 12 with value: 0.8053809218117539.
Best Logistic Regression AUC: 0.80538

OPTIMIZATION COMPLETE


In [7]:
# ============================================================================
# STEP 5: Train Base Models with Optimized Hyperparameters
# ============================================================================
print("\n" + "="*80)
print("TRAINING BASE MODELS")
print("="*80)

# Model 1: XGBoost
print("\n[1/3] Training XGBoost...")
xgb_model = xgb.XGBClassifier(**best_xgb_params)
xgb_model.fit(X_selected, y_resampled)
xgb_preds_train = xgb_model.predict_proba(X_selected)[:, 1]
xgb_preds_test = xgb_model.predict_proba(X_test_selected)[:, 1]
xgb_auc = roc_auc_score(y_resampled, xgb_preds_train)
print(f"XGBoost Train AUC: {xgb_auc:.5f}")

# Model 2: ExtraTrees
print("\n[2/3] Training ExtraTrees...")
et_model = ExtraTreesClassifier(**best_et_params)
et_model.fit(X_selected, y_resampled)
et_preds_train = et_model.predict_proba(X_selected)[:, 1]
et_preds_test = et_model.predict_proba(X_test_selected)[:, 1]
et_auc = roc_auc_score(y_resampled, et_preds_train)
print(f"ExtraTrees Train AUC: {et_auc:.5f}")

# Model 3: Logistic Regression (with calibration)
print("\n[3/3] Training Logistic Regression (with calibration)...")
lr_base = LogisticRegression(**best_lr_params)
lr_model = CalibratedClassifierCV(lr_base, cv=5, method='isotonic')
lr_model.fit(X_selected, y_resampled)
lr_preds_train = lr_model.predict_proba(X_selected)[:, 1]
lr_preds_test = lr_model.predict_proba(X_test_selected)[:, 1]
lr_auc = roc_auc_score(y_resampled, lr_preds_train)
print(f"Logistic Regression Train AUC: {lr_auc:.5f}")

print("\n" + "="*80)
print("BASE MODEL TRAINING COMPLETE")
print("="*80)



TRAINING BASE MODELS

[1/3] Training XGBoost...
XGBoost Train AUC: 0.92665

[2/3] Training ExtraTrees...
ExtraTrees Train AUC: 1.00000

[3/3] Training Logistic Regression (with calibration)...
Logistic Regression Train AUC: 0.62811

BASE MODEL TRAINING COMPLETE


In [8]:
# ============================================================================
# STEP 6: Stacking Ensemble with Meta-Learner
# ============================================================================
print("\n" + "="*80)
print("STACKING ENSEMBLE - TRAINING META-LEARNER")
print("="*80)

# Stack predictions as new features
meta_train = np.column_stack([xgb_preds_train, et_preds_train, lr_preds_train])
meta_test = np.column_stack([xgb_preds_test, et_preds_test, lr_preds_test])

# Train meta-learner (Logistic Regression with regularization)
meta_model = LogisticRegression(C=0.5, penalty='l2', random_state=42, max_iter=1000)
meta_model.fit(meta_train, y_resampled)
meta_preds_train = meta_model.predict_proba(meta_train)[:, 1]
meta_preds_test = meta_model.predict_proba(meta_test)[:, 1]

stacking_auc = roc_auc_score(y_resampled, meta_preds_train)
print(f"Stacking Ensemble Train AUC: {stacking_auc:.5f}")
print(f"\nMeta-learner coefficients:")
print(f"  XGBoost weight: {meta_model.coef_[0][0]:.4f}")
print(f"  ExtraTrees weight: {meta_model.coef_[0][1]:.4f}")
print(f"  Logistic Regression weight: {meta_model.coef_[0][2]:.4f}")



STACKING ENSEMBLE - TRAINING META-LEARNER
Stacking Ensemble Train AUC: 1.00000

Meta-learner coefficients:
  XGBoost weight: -1.5257
  ExtraTrees weight: 16.5152
  Logistic Regression weight: -1.5019


In [9]:
# ============================================================================
# STEP 7: Weighted Averaging Ensemble (Based on CV Performance)
# ============================================================================
print("\n" + "="*80)
print("WEIGHTED AVERAGING ENSEMBLE")
print("="*80)

# Calculate weights based on individual model performance
weights_sum = xgb_auc + et_auc + lr_auc
weights = [xgb_auc/weights_sum, et_auc/weights_sum, lr_auc/weights_sum]

weighted_preds_test = (weights[0] * xgb_preds_test + 
                        weights[1] * et_preds_test + 
                        weights[2] * lr_preds_test)

weighted_preds_train = (weights[0] * xgb_preds_train + 
                        weights[1] * et_preds_train + 
                        weights[2] * lr_preds_train)

weighted_auc = roc_auc_score(y_resampled, weighted_preds_train)
print(f"Weighted Ensemble Train AUC: {weighted_auc:.5f}")
print(f"\nModel weights:")
print(f"  XGBoost: {weights[0]:.4f} ({weights[0]*100:.1f}%)")
print(f"  ExtraTrees: {weights[1]:.4f} ({weights[1]*100:.1f}%)")
print(f"  Logistic Regression: {weights[2]:.4f} ({weights[2]*100:.1f}%)")



WEIGHTED AVERAGING ENSEMBLE
Weighted Ensemble Train AUC: 0.99143

Model weights:
  XGBoost: 0.3627 (36.3%)
  ExtraTrees: 0.3914 (39.1%)
  Logistic Regression: 0.2459 (24.6%)


In [10]:
# ============================================================================
# STEP 8: Final Ensemble (Blend Stacking + Weighted)
# ============================================================================
print("\n" + "="*80)
print("FINAL ENSEMBLE PREDICTIONS")
print("="*80)

# Optimize blending ratio using cross-validation
print("\nOptimizing blending ratio...")
blend_ratios = np.linspace(0.3, 0.8, 11)
best_blend_ratio = 0.65
best_blend_auc = 0

for ratio in blend_ratios:
    blended_preds = ratio * meta_preds_train + (1 - ratio) * weighted_preds_train
    blended_auc = roc_auc_score(y_resampled, blended_preds)
    if blended_auc > best_blend_auc:
        best_blend_auc = blended_auc
        best_blend_ratio = ratio

print(f"Best blending ratio: {best_blend_ratio:.3f} (stacking : weighted)")
print(f"Best blended AUC: {best_blend_auc:.5f}")

# Final predictions
final_preds_test = best_blend_ratio * meta_preds_test + (1 - best_blend_ratio) * weighted_preds_test
final_preds_train = best_blend_ratio * meta_preds_train + (1 - best_blend_ratio) * weighted_preds_train

final_auc = roc_auc_score(y_resampled, final_preds_train)
print(f"\nFinal Ensemble Train AUC: {final_auc:.5f}")
print(f"Prediction range: [{final_preds_test.min():.4f}, {final_preds_test.max():.4f}]")
print(f"Mean prediction: {final_preds_test.mean():.4f}")



FINAL ENSEMBLE PREDICTIONS

Optimizing blending ratio...
Best blending ratio: 0.400 (stacking : weighted)
Best blended AUC: 1.00000

Final Ensemble Train AUC: 1.00000
Prediction range: [0.0695, 0.8806]
Mean prediction: 0.3939


In [11]:
# ============================================================================
# STEP 9: Cross-Validation Evaluation on Original Unbalanced Data
# ============================================================================
print("\n" + "="*80)
print("CROSS-VALIDATION EVALUATION")
print("="*80)

# Evaluate on original unbalanced data using 5-fold CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def cv_predictions(model_class, model_params, X_data, y_data):
    """Get out-of-fold predictions"""
    oof_preds = np.zeros(len(y_data))
    for train_idx, val_idx in cv.split(X_data, y_data):
        X_train_fold = X_data.iloc[train_idx].values if isinstance(X_data, pd.DataFrame) else X_data[train_idx]
        X_val_fold = X_data.iloc[val_idx].values if isinstance(X_data, pd.DataFrame) else X_data[val_idx]
        y_train_fold, y_val_fold = y_data.iloc[train_idx], y_data.iloc[val_idx]
        
        # Create selector for this fold
        selector_fold = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, 
                                        random_state=42, eval_metric='auc', tree_method='hist')
        selector_fold.fit(X_train_fold, y_train_fold)
        selector_transform = SelectFromModel(selector_fold, max_features=150, threshold=-np.inf)
        
        # Apply SMOTE to training fold
        smote_fold = SMOTE(sampling_strategy=0.5, random_state=42, k_neighbors=5)
        X_train_smote, y_train_smote = smote_fold.fit_resample(X_train_fold, y_train_fold)
        
        # Transform with fold-specific selector
        X_train_selected = selector_transform.transform(X_train_smote)
        X_val_selected = selector_transform.transform(X_val_fold)
        
        # Train and predict
        model = model_class(**model_params)
        model.fit(X_train_selected, y_train_smote)
        oof_preds[val_idx] = model.predict_proba(X_val_selected)[:, 1]
    
    return oof_preds

print("\nComputing out-of-fold predictions for ensemble...")
print("(This may take a few minutes)")

# Get OOF predictions for each model (pass class and params instead of fitted model)
print("Computing OOF for XGBoost...")
xgb_oof = cv_predictions(xgb.XGBClassifier, best_xgb_params, X, y)

print("Computing OOF for ExtraTrees...")
et_oof = cv_predictions(ExtraTreesClassifier, best_et_params, X, y)

print("Computing OOF for Logistic Regression...")
lr_oof = cv_predictions(LogisticRegression, best_lr_params, X, y)

# Stack and blend OOF predictions
meta_oof = np.column_stack([xgb_oof, et_oof, lr_oof])
meta_oof_preds = meta_model.predict_proba(meta_oof)[:, 1]

weighted_oof = weights[0] * xgb_oof + weights[1] * et_oof + weights[2] * lr_oof
final_oof = best_blend_ratio * meta_oof_preds + (1 - best_blend_ratio) * weighted_oof

# Calculate metrics
threshold = 0.5
final_oof_binary = (final_oof >= threshold).astype(int)

cv_auc = roc_auc_score(y, final_oof)
cv_f1 = f1_score(y, final_oof_binary)
cv_precision = precision_score(y, final_oof_binary)
cv_recall = recall_score(y, final_oof_binary)

print(f"\nCross-Validated Metrics (5-fold):")
print(f"  AUC: {cv_auc:.5f}")
print(f"  F1 Score: {cv_f1:.5f}")
print(f"  Precision: {cv_precision:.5f}")
print(f"  Recall: {cv_recall:.5f}")



CROSS-VALIDATION EVALUATION

Computing out-of-fold predictions for ensemble...
(This may take a few minutes)
Computing OOF for XGBoost...
Computing OOF for ExtraTrees...
Computing OOF for Logistic Regression...

Cross-Validated Metrics (5-fold):
  AUC: 0.83094
  F1 Score: 0.52235
  Precision: 0.60933
  Recall: 0.45711


In [12]:
# ============================================================================
# STEP 10: Generate Submission File
# ============================================================================
print("\n" + "="*80)
print("GENERATING SUBMISSION FILE")
print("="*80)

submission = pd.DataFrame({
    'claim_number': test_original['claim_number'],
    'subrogation': final_preds_test
})

submission.to_csv('baseline_v2_submission.csv', index=False)
print("Submission file saved: baseline_v2_submission.csv")
print(f"\nSubmission statistics:")
print(f"  Shape: {submission.shape}")
print(f"  Prediction range: [{submission['subrogation'].min():.4f}, {submission['subrogation'].max():.4f}]")
print(f"  Mean prediction: {submission['subrogation'].mean():.4f}")
print(f"\nFirst 10 predictions:")
print(submission.head(10))
print(f"\nLast 10 predictions:")
print(submission.tail(10))

# Also save binary version (threshold = 0.5)
submission_binary = submission.copy()
submission_binary['subrogation'] = (submission_binary['subrogation'] >= 0.5).astype(int)
submission_binary.to_csv('baseline_v2_submission_binary.csv', index=False)
print(f"\nBinary submission saved: baseline_v2_submission_binary.csv")



GENERATING SUBMISSION FILE
Submission file saved: baseline_v2_submission.csv

Submission statistics:
  Shape: (12000, 2)
  Prediction range: [0.0695, 0.8806]
  Mean prediction: 0.3939

First 10 predictions:
   claim_number  subrogation
0       3126034     0.272111
1       7380142     0.440979
2       4655051     0.216919
3       6728725     0.787726
4       9848460     0.318639
5       7164028     0.111893
6       8693447     0.584767
7       3185825     0.337260
8       4469488     0.327042
9       6027831     0.731881

Last 10 predictions:
       claim_number  subrogation
11990       8550044     0.834583
11991       6752685     0.151940
11992       8890691     0.379889
11993       2253734     0.645585
11994       3618383     0.752167
11995       3760534     0.183342
11996       9934539     0.614633
11997       1682245     0.137872
11998       6431119     0.592145
11999       7730901     0.284031

Binary submission saved: baseline_v2_submission_binary.csv


In [13]:
# ============================================================================
# STEP 11: Feature Importance Analysis
# ============================================================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80)

# Combine feature importances from tree models
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'xgb_importance': xgb_model.feature_importances_,
    'et_importance': et_model.feature_importances_
})

# Average importance across tree models
feature_importance['avg_importance'] = feature_importance[['xgb_importance', 'et_importance']].mean(axis=1)
feature_importance = feature_importance.sort_values('avg_importance', ascending=False)

print("\nTop 30 Most Important Features:")
print(feature_importance.head(30).to_string(index=False))

feature_importance.to_csv('baseline_v2_feature_importance.csv', index=False)
print(f"\nFeature importance saved: baseline_v2_feature_importance.csv")



FEATURE IMPORTANCE ANALYSIS

Top 30 Most Important Features:
                                       feature  xgb_importance  et_importance  avg_importance
                             accident_type_enc        0.087821       0.063116        0.075468
           liab_prct_x_witness_present_ind_enc        0.088661       0.024553        0.056607
                                liab_prct_rank        0.054810       0.054489        0.054650
                        accident_site_enc_freq        0.062112       0.032898        0.047505
              liab_prct_plus_accident_type_enc        0.056270       0.029919        0.043094
                            multi_vehicle_flag        0.036124       0.041363        0.038744
                  accident_site_enc_target_enc        0.048219       0.028423        0.038321
                            high_education_ind        0.025423       0.028839        0.027131
                                     liab_prct        0.023382       0.030099        0.02674

In [14]:
# ============================================================================
# SUMMARY
# ============================================================================
print("\n" + "="*80)
print("BASELINE VERSION 2 - COMPLETE!")
print("="*80)
print("\nMODELS USED:")
print("✓ 1. XGBoost (Gradient Boosting)")
print("✓ 2. ExtraTrees (Bagging)")
print("✓ 3. Logistic Regression with Calibration (Linear)")
print("\nTECHNIQUES APPLIED:")
print("✓ SMOTE for class imbalance (50% minority)")
print("✓ Feature selection (top 150 features)")
print("✓ Bayesian hyperparameter optimization (Optuna)")
print("✓ Stacking with Logistic Regression meta-learner")
print("✓ Weighted averaging based on performance")
print("✓ Optimized blending ratio")
print("✓ 5-fold stratified cross-validation")
print("\nOUTPUT FILES:")
print("- baseline_v2_submission.csv (probabilities)")
print("- baseline_v2_submission_binary.csv (binary predictions)")
print("- baseline_v2_feature_importance.csv (feature analysis)")
print("\nMODEL DIVERSITY:")
print("✓ Algorithmic diversity: Boosting (XGB) + Bagging (ET) + Linear (LR)")
print("✓ Complementary strengths for robust ensemble")
print("="*80)



BASELINE VERSION 2 - COMPLETE!

MODELS USED:
✓ 1. XGBoost (Gradient Boosting)
✓ 2. ExtraTrees (Bagging)
✓ 3. Logistic Regression with Calibration (Linear)

TECHNIQUES APPLIED:
✓ SMOTE for class imbalance (50% minority)
✓ Feature selection (top 150 features)
✓ Bayesian hyperparameter optimization (Optuna)
✓ Stacking with Logistic Regression meta-learner
✓ Weighted averaging based on performance
✓ Optimized blending ratio
✓ 5-fold stratified cross-validation

OUTPUT FILES:
- baseline_v2_submission.csv (probabilities)
- baseline_v2_submission_binary.csv (binary predictions)
- baseline_v2_feature_importance.csv (feature analysis)

MODEL DIVERSITY:
✓ Algorithmic diversity: Boosting (XGB) + Bagging (ET) + Linear (LR)
✓ Complementary strengths for robust ensemble
